In [1]:
using Random
using Distributions
using PyPlot
using QuantumRelay
using DataFrames
using RobustAdaptiveMetropolisSampler
using LinearAlgebra
function qrelay(alpha, delta, name)
    n = 6
    chi = fill(sqrt(0.06), n)
    phi = [im * tanh(chi[i]) for i=1:n]
    omega = [1.0 / prod(cosh(chi[j]))^2 for j=1:n]
    syms, op = qrelay_op(n, phi, alpha, delta)
    op_a, op_ab, mat, coef = op_mat(op)

    op_q2 = [syms.apH[1], syms.apV[1], syms.bpH[end], syms.bpV[end]]
    op_q1 = [syms.apH[2:end]..., syms.apV[2:end]..., syms.bpH[1:end-1]..., syms.bpV[1:end-1]...]
    mask_q1 = [op in op_q1 for op in op_a];
    mask_q2 = [op in op_q2 for op in op_a];
    qq = [x in syms.apH || x in syms.bpV ? 1 : 0 for x in op_a]
    
    pdet0 = pdet_maker(0.04, 1e-5)
    qrs = QRelaySampler(mat, coef, omega, pdet0)

    targetcache = Dict{Vector{Int}, Float64}()
    function plogtarget(na::Vector{Int})
        get!(targetcache, na) do
            log(qrs.prob(qq, na, mask_q1) * qrs.prob(na))
        end
    end
    chain, accrate, S = RAM_sample(p -> logpdf(plogtarget,p[1]), zeros(length(qq)),0.5, 1000)
     
    
    funcQ(v)=qrs.prob(qq, v, mask_q2)
    return qrs, chain, accrate ,S, funcQ
end
        

┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /Users/midow/.julia/packages/Gadfly/09PWZ/src/mapping.jl:228


qrelay (generic function with 1 method)

In [ ]:
dataname="data3"
results = []
for i = 0:12
    beta = i*pi/12
    name = string(i) 
    mkpath("$dataname/$name")
    qrs, chain, accrate, S , funcQ = qrelay(pi/4, beta, name)
    println("beta:", beta)

    push!(results, (qrs, chain, accrate, S, funcQ))
end